<a href="https://colab.research.google.com/github/jonas2024dev/jonas2024dev/blob/main/ETLbasico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
!pip install pyspark

In [93]:
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import IntegerType,StringType,StructType,StructField
from pyspark.sql.functions import col,lit

from typing_extensions import dataclass_transform



inicio sesion spark

In [94]:
spark=SparkSession.builder.appName('ETLBSC').getOrCreate()
sqlContext = SQLContext(spark.sparkContext) # Create an instance using SparkSession's SparkContext


/usr/local/lib/python3.11/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [95]:
path = "/content/drive/MyDrive/"
players_info=spark.read.csv(path+ "players_info.csv")


In [96]:
display(players_info)

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string, _c16: string, _c17: string, _c18: string, _c19: string, _c20: string, _c21: string, _c22: string, _c23: string, _c24: string, _c25: string]

In [97]:
players_info.show()


+----+--------------------+---+------+-----------------+------------------+-------------+--------+--------------+--------------+--------------+------------+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+--------------------+
| _c0|                 _c1|_c2|   _c3|              _c4|               _c5|          _c6|     _c7|           _c8|           _c9|          _c10|        _c11|_c12|_c13| _c14|_c15| _c16|_c17| _c18|_c19| _c20|_c21| _c22|_c23| _c24|                _c25|
+----+--------------------+---+------+-----------------+------------------+-------------+--------+--------------+--------------+--------------+------------+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+--------------------+
|rank|         player_name|age|gender|             club|            league|  nationality|position|preffered_foot|height_(in cm)|weight_(in kg)|alt_position| ovr| pac|pac+-| sho|sho+-| pas|pas+-| dri|dri+-| def|def+-| phy|phy+-|           image_url|
|   

In [98]:
players_info1=players_info.select('_c1','_c2','_c4','_c7','_c12','_c13','_c15','_c17','_c19','_c21','_c23')

In [99]:
players_info1.show()


+--------------------+---+-----------------+--------+----+----+----+----+----+----+----+
|                 _c1|_c2|              _c4|     _c7|_c12|_c13|_c15|_c17|_c19|_c21|_c23|
+--------------------+---+-----------------+--------+----+----+----+----+----+----+----+
|         player_name|age|             club|position| ovr| pac| sho| pas| dri| def| phy|
|       Kylian Mbappé| 25|      Real Madrid|      ST|  91|  97|  90|  80|  92|  36|  78|
|               Rodri| 28|  Manchester City|     CDM|  91|  66|  80|  86|  84|  87|  85|
|      Aitana Bonmatí| 26|     FC Barcelona|      CM|  91|  81|  86|  86|  91|  77|  75|
|      Erling Haaland| 24|  Manchester City|      ST|  91|  88|  92|  70|  81|  45|  88|
|     Jude Bellingham| 21|      Real Madrid|     CAM|  90|  80|  87|  83|  88|  78|  83|
|     Alexia Putellas| 30|     FC Barcelona|      CM|  90|  82|  89|  90|  91|  72|  78|
|            Vini Jr.| 24|      Real Madrid|      LW|  90|  95|  84|  81|  91|  29|  69|
|Caroline Graham H...

In [100]:
def rename_columns(index,iterator):
  # Skip the first row (header) only for the first partition (index 0)
  if index == 0:
    iterator = iter(list(iterator)[1:])
  for row in iterator:
    # Convert relevant columns to integers, handling potential errors
    try:
      row_list = list(row)
      row_list[1] = int(row_list[1]) # age
      row_list[4] = int(row_list[4]) # ovr
      row_list[5] = int(row_list[5]) # pac
      row_list[6] = int(row_list[6]) # sho
      row_list[7] = int(row_list[7]) # pas
      row_list[8] = int(row_list[8]) # dri
      row_list[9] = int(row_list[9]) # def
      row_list[10] = int(row_list[10]) # phy
      yield tuple(row_list)
    except ValueError:
      # Handle rows with invalid data, e.g., skip them or replace with None
      print(f"Skipping row due to invalid data: {row}")
      pass  # Skip this row





In [101]:
rdd=players_info1.rdd



In [102]:
rdd.collect()


[Row(_c1='player_name', _c2='age', _c4='club', _c7='position', _c12='ovr', _c13='pac', _c15='sho', _c17='pas', _c19='dri', _c21='def', _c23='phy'),
 Row(_c1='Kylian Mbappé', _c2='25', _c4='Real Madrid', _c7='ST', _c12='91', _c13='97', _c15='90', _c17='80', _c19='92', _c21='36', _c23='78'),
 Row(_c1='Rodri', _c2='28', _c4='Manchester City', _c7='CDM', _c12='91', _c13='66', _c15='80', _c17='86', _c19='84', _c21='87', _c23='85'),
 Row(_c1='Aitana Bonmatí', _c2='26', _c4='FC Barcelona', _c7='CM', _c12='91', _c13='81', _c15='86', _c17='86', _c19='91', _c21='77', _c23='75'),
 Row(_c1='Erling Haaland', _c2='24', _c4='Manchester City', _c7='ST', _c12='91', _c13='88', _c15='92', _c17='70', _c19='81', _c21='45', _c23='88'),
 Row(_c1='Jude Bellingham', _c2='21', _c4='Real Madrid', _c7='CAM', _c12='90', _c13='80', _c15='87', _c17='83', _c19='88', _c21='78', _c23='83'),
 Row(_c1='Alexia Putellas', _c2='30', _c4='FC Barcelona', _c7='CM', _c12='90', _c13='82', _c15='89', _c17='90', _c19='91', _c21='7

In [103]:
rdd=rdd.mapPartitionsWithIndex(rename_columns)

In [104]:
rdd.collect()

[('Kylian Mbappé', 25, 'Real Madrid', 'ST', 91, 97, 90, 80, 92, 36, 78),
 ('Rodri', 28, 'Manchester City', 'CDM', 91, 66, 80, 86, 84, 87, 85),
 ('Aitana Bonmatí', 26, 'FC Barcelona', 'CM', 91, 81, 86, 86, 91, 77, 75),
 ('Erling Haaland', 24, 'Manchester City', 'ST', 91, 88, 92, 70, 81, 45, 88),
 ('Jude Bellingham', 21, 'Real Madrid', 'CAM', 90, 80, 87, 83, 88, 78, 83),
 ('Alexia Putellas', 30, 'FC Barcelona', 'CM', 90, 82, 89, 90, 91, 72, 78),
 ('Vini Jr.', 24, 'Real Madrid', 'LW', 90, 95, 84, 81, 91, 29, 69),
 ('Caroline Graham Hansen',
  29,
  'FC Barcelona',
  'RW',
  90,
  89,
  87,
  88,
  90,
  47,
  76),
 ('Kevin De Bruyne', 33, 'Manchester City', 'CM', 90, 67, 87, 94, 87, 65, 78),
 ('Sam Kerr', 31, 'Chelsea', 'ST', 90, 85, 88, 74, 90, 42, 83),
 ('Harry Kane', 31, 'FC Bayern München', 'ST', 90, 65, 93, 84, 83, 49, 82),
 ('Martin Ødegaard', 25, 'Arsenal', 'CM', 89, 70, 82, 89, 89, 67, 66),
 ('Gianluigi Donnarumma', 25, 'Paris SG', 'GK', 89, 90, 84, 73, 90, 52, 87),
 ('Sophia Smit

In [105]:
schema=StructType([

    StructField('player_name',StringType(),True),
    StructField('age',IntegerType(),True),
    StructField('club',StringType(),True),
    StructField('position',StringType(),True),
    StructField('ovr',IntegerType(),True),
    StructField('pac',IntegerType(),True),
    StructField('sho',IntegerType(),True),
    StructField('pas',IntegerType(),True),
    StructField('dri',IntegerType(),True),
    StructField('def',IntegerType(),True),
    StructField('phy',IntegerType(),True),
     ])

In [106]:

dataframe=sqlContext.createDataFrame(rdd,schema)
dataframe.printSchema()


root
 |-- player_name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- club: string (nullable = true)
 |-- position: string (nullable = true)
 |-- ovr: integer (nullable = true)
 |-- pac: integer (nullable = true)
 |-- sho: integer (nullable = true)
 |-- pas: integer (nullable = true)
 |-- dri: integer (nullable = true)
 |-- def: integer (nullable = true)
 |-- phy: integer (nullable = true)



In [107]:
display(dataframe)

DataFrame[player_name: string, age: int, club: string, position: string, ovr: int, pac: int, sho: int, pas: int, dri: int, def: int, phy: int]

In [108]:
dataframe.show()

+--------------------+---+-----------------+--------+---+---+---+---+---+---+---+
|         player_name|age|             club|position|ovr|pac|sho|pas|dri|def|phy|
+--------------------+---+-----------------+--------+---+---+---+---+---+---+---+
|       Kylian Mbappé| 25|      Real Madrid|      ST| 91| 97| 90| 80| 92| 36| 78|
|               Rodri| 28|  Manchester City|     CDM| 91| 66| 80| 86| 84| 87| 85|
|      Aitana Bonmatí| 26|     FC Barcelona|      CM| 91| 81| 86| 86| 91| 77| 75|
|      Erling Haaland| 24|  Manchester City|      ST| 91| 88| 92| 70| 81| 45| 88|
|     Jude Bellingham| 21|      Real Madrid|     CAM| 90| 80| 87| 83| 88| 78| 83|
|     Alexia Putellas| 30|     FC Barcelona|      CM| 90| 82| 89| 90| 91| 72| 78|
|            Vini Jr.| 24|      Real Madrid|      LW| 90| 95| 84| 81| 91| 29| 69|
|Caroline Graham H...| 29|     FC Barcelona|      RW| 90| 89| 87| 88| 90| 47| 76|
|     Kevin De Bruyne| 33|  Manchester City|      CM| 90| 67| 87| 94| 87| 65| 78|
|            Sam

In [109]:
dataframe.dropna()


DataFrame[player_name: string, age: int, club: string, position: string, ovr: int, pac: int, sho: int, pas: int, dri: int, def: int, phy: int]

In [110]:
dataframe.printSchema()

root
 |-- player_name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- club: string (nullable = true)
 |-- position: string (nullable = true)
 |-- ovr: integer (nullable = true)
 |-- pac: integer (nullable = true)
 |-- sho: integer (nullable = true)
 |-- pas: integer (nullable = true)
 |-- dri: integer (nullable = true)
 |-- def: integer (nullable = true)
 |-- phy: integer (nullable = true)



In [121]:
dataframe=dataframe.withColumn('Mean',((col('pac')+col('sho')+col('pas')+col('dri')+col('def')+col('phy'))/lit(6)))
dataframe = dataframe.withColumn("promedio", col("Mean").cast("int"))
dataframe=dataframe.drop('Mean')


In [122]:
display(dataframe)

DataFrame[player_name: string, age: int, club: string, position: string, ovr: int, pac: int, sho: int, pas: int, dri: int, def: int, phy: int, promedio: int]

In [123]:
dataframe.show(100)

+--------------------+---+------------------+--------+---+---+---+---+---+---+---+--------+
|         player_name|age|              club|position|ovr|pac|sho|pas|dri|def|phy|promedio|
+--------------------+---+------------------+--------+---+---+---+---+---+---+---+--------+
|       Kylian Mbappé| 25|       Real Madrid|      ST| 91| 97| 90| 80| 92| 36| 78|      78|
|               Rodri| 28|   Manchester City|     CDM| 91| 66| 80| 86| 84| 87| 85|      81|
|      Aitana Bonmatí| 26|      FC Barcelona|      CM| 91| 81| 86| 86| 91| 77| 75|      82|
|      Erling Haaland| 24|   Manchester City|      ST| 91| 88| 92| 70| 81| 45| 88|      77|
|     Jude Bellingham| 21|       Real Madrid|     CAM| 90| 80| 87| 83| 88| 78| 83|      83|
|     Alexia Putellas| 30|      FC Barcelona|      CM| 90| 82| 89| 90| 91| 72| 78|      83|
|            Vini Jr.| 24|       Real Madrid|      LW| 90| 95| 84| 81| 91| 29| 69|      74|
|Caroline Graham H...| 29|      FC Barcelona|      RW| 90| 89| 87| 88| 90| 47| 7

In [124]:
dataframe.repartition(1).write.csv('output.csv',sep=',',mode='overwrite')

In [125]:
spark.read.csv('output.csv')

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string]